### AIR Project

In [69]:
import string
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from gensim.models import Word2Vec
import multiprocessing
from time import time

In [70]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

def pre_process():
    data = pd.read_csv('WELFake_Dataset.csv', index_col=0)
    print(data.shape)
    # display(data[:300])
    for i,x in data.iterrows():
        if len(str(x["text"])) <= 10:
            data.loc[i, "text"] = np.nan
        if len(str(x["title"])) <= 10:
            data.loc[i, "title"] = np.nan

    data.dropna(inplace=True)
    print(data.shape)
    data.reset_index(drop=True, inplace=True)
    data.to_csv("data/data.csv")
    display(data[:300])

def tokenize():
    stop = stopwords.words('english')
    stemmer = SnowballStemmer('english')
    punc = [u'\u201c',u'\u201d',u'\u2018',u'\u2019',u'\u2024',u'\u2025',u'\u2026',u'\u2027']
    # print(punc)
    data = pd.read_csv('data/data.csv', index_col=0)
    data_cleaned = data.copy()
    titles = list()
    texts = list()
    for i, row in data.iterrows():
        title = str(row["title"])
        text = str(row["text"])
        t1 = ""
        for c in title:
            if not (c in string.punctuation or c in punc):
                t1 += c
            else:
                t1 += " "
        t2 = ""
        for c in text:
            if not (c in string.punctuation or c in punc):
                t2 += c
            else:
                t2 += " "
        title_tokens = nltk.tokenize.word_tokenize(t1)
        text_tokens = nltk.tokenize.word_tokenize(t2)
        # title_filtered = [w.lower() for w in title_tokens if not w.lower() in string.punctuation]
        # title_filtered = [w.lower() for w in title_filtered if not w.lower() in punc]
        title_filtered = [w.lower() for w in title_tokens if not w.lower() in stop]
        title_stemmed = [stemmer.stem(w) for w in title_filtered]
        # text_filtered = [w.lower() for w in text_tokens if not w.lower() in string.punctuation]
        # text_filtered = [w.lower() for w in text_filtered if not w.lower() in punc]
        text_filtered = [w.lower() for w in text_tokens if not w.lower() in stop]
        text_stemmed = [stemmer.stem(w) for w in text_filtered]
        # print(title_stemmed)
        # print(text_stemmed)
        titles.append(title_stemmed)
        texts.append(text_stemmed)
        if i % 5000 == 0:
            print(i)
    data_cleaned["title"] = titles
    data_cleaned["text"] = texts
    data_cleaned.to_csv("data/data_token.csv")

In [71]:
load_model_from_disc = True
w2v_model = None
if load_model_from_disc:
    try:
        w2v_model = Word2Vec.load("word2vec.model")
    except:
        pass

if w2v_model is None or not load_model_from_disc:
    if load_model_from_disc:
        print("Could not load model from disc. Training model...")
    else:
        print("Loading from disc deactivated. Training model...")
    data = pd.read_csv('data_tokenized/data_token.csv', index_col=0, dtype=str)

    class MySentences(object):
        def __init__(self, data):
            self.data = data

        def __iter__(self):
            for doc in data["text"]: #change to "title" or combine both
                words = doc.split(",")
                doc = []
                for word in words:
                    doc.append(str(word).replace("'", "").replace(" ", "").replace("[", "").replace("]", ""))
                yield doc

    sentences = MySentences(data)

    cores = multiprocessing.cpu_count()
    w2v_model = Word2Vec(min_count=20,
                         window=2,
                         sample=6e-5,
                         alpha=0.03,
                         min_alpha=0.0007,
                         negative=20,
                         workers=cores-1)

    w2v_model.build_vocab(sentences, progress_per=10000)
    t = time()
    w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=3, report_delay=1)
    print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))
    w2v_model.save("word2vec.model")
else:
    print("Model loaded from disc.")


Loading from disc deactivated. Training model...
Time to train the model: 2.21 mins


In [72]:
# calculate similarity
w2v_model.wv.similarity("amazon", 'nazi')

-0.043702327

In [73]:
# calculate similarity
w2v_model.wv.similarity("obama", 'trump')

0.604767

In [74]:
# find out which element doesn't match
w2v_model.wv.doesnt_match(['amazon', 'obama', 'trump'])

'amazon'

In [75]:
# Which word is to obama as georg is to bush?
w2v_model.wv.most_similar(positive=["obama", "georg"], negative=["bush"], topn=3)

[('barack', 0.6333215236663818),
 ('outgo', 0.43497365713119507),
 ('presid', 0.404074490070343)]

In [76]:
# e.g. words most similar to obama
w2v_model.wv.most_similar(positive=["obama"])

[('barack', 0.8282706141471863),
 ('presid', 0.667381227016449),
 ('predecessor', 0.6564348936080933),
 ('outgo', 0.6217739582061768),
 ('administr', 0.6193204522132874),
 ('trump', 0.6047670245170593),
 ('undo', 0.585784375667572),
 ('bush', 0.5590077042579651),
 ('bachelet', 0.5531689524650574),
 ('michell', 0.5389623045921326)]

In [77]:
# e.g. words most similar to obama
w2v_model.wv.most_similar(positive=["presid"])

[('obama', 0.6673811078071594),
 ('barack', 0.6597641706466675),
 ('45th', 0.6460375189781189),
 ('outgo', 0.6316460967063904),
 ('predecessor', 0.630837619304657),
 ('successor', 0.61188805103302),
 ('administr', 0.6103466749191284),
 ('donald', 0.6015743017196655),
 ('presidenti', 0.6010771989822388),
 ('trump', 0.5971187353134155)]